# EC2 User data / Metadata

## EC2 User Data
- 인스턴스의 최초 실행 시 지정한 스크립트 실행 가능
    - 별도 설정을 통해 재부팅 마다 실행하도록 설정 가능
- 두 가지 모드
    - Shell Script
    - Cloud-init : 리눅스 이미지의 부트스트래핑을 위한 오픈소스 어플리케이션
- 주요 사용 사례
    - 인스턴스 설정
    - 외부 패키지 다운로드
    - 설치 어플리케이션 실행
    - 기타 인스턴스 실행 시 필요한 동작

### 예제 - 인스턴스 등록
- 인스턴스가 실행될 때 마다, 인스턴스의 정보를 모아두고 싶다면? (인스턴스 아이디, 아이피, AMI ID등)
    - 유저데이터를 통해 구현 가능! 그런데, 인스턴스의 정보는 어떻게 얻을 수 있지?
        - Amazon EC2 Instance Metadata를 통해 받아올 수 있다

## Amazon EC2 Instance Metadata
- 인스턴스 메타데이터는 실행 중인 인스턴스를 구성 또는 관리하는데 사용될 수 있는 인스턴스 관련 데이터 입니다. 인스턴스 메타데이터는 호스트 이름, 이벤트 및 보안 그룹과 같은 범주로 분류됩니다.
- 인스턴스의 속성 및 정보 데이터
    - AMI ID, IPv4주소, EBS 매핑, 보안 그룹 연동 상황 등
- 실행 중인 EC2 인스턴스의 메타데이터 IMDS(Instance Metadata Service)로 조회 가능
    - HTTP 엔드포인트 지원
    - IPv4주소 : 169.254.169.254 (외우자!)
    - IPv6주소 : fd00:ec2::254
    - 두 가지 모드
        - IMDS v1 : Request / Response 기반
        - IMDS v2 : 세션 기반 (default)
- 주요 사용 사례
    - 인스턴스 별 설정, IAM 임시 자격증명 조회 등 (AWS CLI, SDK등이 내부적으로 활용)
- EC2 실행 시 메타데이터 액세스 가능 여부 설정
    - 기본적으로 활성화
    - 버전 선택 가능 (v2가 디폴트)
- 가격은 무려
- 참고로 Tag 조회의 경우 별도로 활성화 해야 Metadata로 Tag조회 가능
    - 활성화 하지 않을 경우 목록에서 보이지 않음

### Instance Metadata Service v1
- 별도의 보안 인증이 필요 없는 Request / Response 기반
    - Link Local IP (169.254.169.254)를 사용하기 떄문에, 해당 EC2 인스턴스에서만 요청 가능
- CloudWatch Metric을 통해 조회 횟수 기록 가능
- 예시
    - 인스턴스 이름 가져오기 : curl http://169.254.169.254/latest/meda-data-/tags/instance/Name
- 보안 상 취약점이 있음

### Instance Metadata Service v2
- 보안 토큰을 발급, 요청 때 마다 토큰을 사용해 인증하는 세션 방식
    - 토큰의 유효 기간은 1초에서 최대 6시간
- IMDS v1 보다 더 높은 보안 수준 제공
    - IAM 정책 등을 활용하면 인스턴스가 IMDS v2만 사용하도록 강제 가능
- 예시
    - TOKEN = 'curl -X PUT "http://169.254.169.254/latest/api/token" -H "X-aws-ec2-metadate-token-ttl-seconds:21600"'
    - curl -H "X-aws-ec2-metadata-token=$TOKEN" -v http://169.254.169.254/latest/meta-data/tags/instance/Name

### 실습
- User Data / Meta data 활용
    - EC2가 올라갈 때 웹 서버 설치 및 기동
    - 이 과정에서 index.html에 자신의 인스턴스 아이디를 업데이트
        - 메타데이터를 활용하여 자신의 인스턴스 아이디 조회하여 index.html에 업데이트
1. 인스턴스 프로비전 - 다른건 기본으로 하면 되는데, 고급 설정에서 메타데이터 액세스 설정을 만져야 한다.(태그 허용 활성화)
2. 인스턴스 연결, 메타데이터 받아오기
    - 토큰 받아오기 : TOKEN=$(curl -X PUT "http://169.254.169.254/latest/api/token" -H "X-aws-ec2-metadata-token-ttl-seconds: 21600")
    - 받아올 수 있는 메타데이터 목록 조회 : curl -s -H "X-aws-ec2-metadata-token: $TOKEN" http://169.254.169.254/latest/meta-data
    - 인스턴스 이름 메타데이터 받아오기 : curl -s -H "X-aws-ec2-metadata-token: $TOKEN" http://169.254.169.254/latest/meta-data/instance-id
    - 인스턴스 태그 데이터 받아오기 : curl -s -H "X-aws-ec2-metadata-token: $TOKEN" http://169.254.169.254/latest/meta-data/tags/instance/Name
3. 이제 실제로 서버 설치 및 기동, 인스턴스 아이디 업데이트를 해보자
    - 인스턴스 프로비전(HTTP 트래픽 허용, 고급 세부 정보에서 사용자 데이터 입력)
        - #!/bin/bash : 배쉬 스크립트 사용
        - sudo -s : 수도 권한
        - dnf install httpd -y : 아파치 설치
        - service httpd start : 아파치 실행
        - chkconfig httpd on : 인스턴스 실행 시 자동으로 아파치가 실행되도록 서비스 등록
        - TOKEN=$(curl -X PUT "http://169.254.169.254/latest/api/token" -H "X-aws-ec2-metadata-token-ttl-seconds: 21600") : 인증 토큰 받아오기
        - INSTANCE_ID=$(curl -s -H "X-aws-ec2-metadata-token: $TOKEN" http://169.254.169.254/latest/meta-data/instance-id) : 인스턴스 아이디를 받아오기
        - echo "$INSTANCE_ID" >> /var/www/html/index.html : 받아온 인스턴스 아이디를 index.html에 넣기
    - 로그 확인
        - cd /var/log
        - nano cloud-init.log : 인스턴스가 실행될 때 로그 기록
        - nano cloud-init-output : 실행 결과가 출력되는 아웃풋 로그 기록. 더 자세한 디버깅 가능